In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import random
from datetime import datetime, timedelta

# Sample data generation
def generate_supplier_data():
    suppliers = [
        "GreenTech Manufacturing", "EcoFabrics Ltd", "SustainablePack Co",
        "CleanEnergy Logistics", "EthicalMaterials Inc", "CarbonNeutral Shipping",
        "RenewableResources Corp", "GreenSupply Chain", "EcoEfficient Systems",
        "SustainableGoods Ltd", "CleanTech Solutions", "GreenLogistics Pro"
    ]

    regions = ["North America", "Europe", "Asia Pacific", "Latin America", "Middle East", "Africa"]

    data = []
    for i, supplier in enumerate(suppliers):
        # Generate realistic but varied sustainability metrics
        base_carbon = random.uniform(50, 200)  # Base carbon footprint
        trend_factor = random.uniform(0.8, 1.2)  # Some suppliers improving, others declining

        data.append({
            "Supplier": supplier,
            "Region": random.choice(regions),
            "Carbon_Footprint_tCO2": round(base_carbon * trend_factor, 1),
            "ESG_Score": random.randint(65, 95),
            "Labor_Compliance": random.choice(["Green", "Yellow", "Red"]),
            "Audit_Date": (datetime.now() - timedelta(days=random.randint(1, 365))).strftime("%Y-%m-%d"),
            "Risk_Level": random.choice(["Low", "Medium", "High"]),
            "Certification": random.choice(["ISO 14001", "B-Corp", "Fair Trade", "LEED", "None"]),
            "Emissions_Trend": random.choice(["Improving", "Stable", "Declining"]),
            "Contract_Value": random.randint(100000, 5000000),
            "Latitude": random.uniform(-60, 60),
            "Longitude": random.uniform(-180, 180),
            "Last_Alert": random.choice([None, "High Carbon Spike", "Audit Overdue", "Compliance Issue"])
        })

    return pd.DataFrame(data)

# Initialize data
df_suppliers = generate_supplier_data()

def create_carbon_footprint_chart(region_filter="All"):
    """Create carbon footprint visualization"""
    filtered_df = df_suppliers if region_filter == "All" else df_suppliers[df_suppliers["Region"] == region_filter]

    fig = px.bar(
        filtered_df,
        x="Supplier",
        y="Carbon_Footprint_tCO2",
        color="Emissions_Trend",
        title="Carbon Footprint by Supplier (tCO2)",
        color_discrete_map={
            "Improving": "#28a745",
            "Stable": "#ffc107",
            "Declining": "#dc3545"
        }
    )
    fig.update_layout(
        xaxis_tickangle=-45,
        height=500,
        showlegend=True
    )
    return fig

def create_esg_scorecard():
    """Create ESG scorecard heatmap"""
    # Create matrix for heatmap
    categories = ["Environmental", "Social", "Governance", "Overall"]
    supplier_sample = df_suppliers["Supplier"].head(8).tolist()

    # Generate ESG scores matrix
    esg_matrix = []
    for supplier in supplier_sample:
        scores = [
            random.randint(60, 100),  # Environmental
            random.randint(65, 95),   # Social
            random.randint(70, 100),  # Governance
            random.randint(65, 95)    # Overall
        ]
        esg_matrix.append(scores)

    fig = go.Figure(data=go.Heatmap(
        z=esg_matrix,
        x=categories,
        y=supplier_sample,
        colorscale="RdYlGn",
        text=esg_matrix,
        texttemplate="%{text}",
        textfont={"size": 12},
        hoverongaps=False
    ))

    fig.update_layout(
        title="ESG Scorecard Matrix",
        height=400,
        xaxis_title="ESG Categories",
        yaxis_title="Suppliers"
    )
    return fig

def create_risk_dashboard():
    """Create risk assessment dashboard"""
    risk_counts = df_suppliers["Risk_Level"].value_counts()

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "domain"}, {"type": "bar"}]],
        subplot_titles=("Risk Distribution", "Compliance Status")
    )

    # Risk pie chart
    fig.add_trace(
        go.Pie(
            labels=risk_counts.index,
            values=risk_counts.values,
            marker_colors=["#28a745", "#ffc107", "#dc3545"],
            name="Risk Levels"
        ),
        row=1, col=1
    )

    # Compliance bar chart
    compliance_counts = df_suppliers["Labor_Compliance"].value_counts()
    fig.add_trace(
        go.Bar(
            x=compliance_counts.index,
            y=compliance_counts.values,
            marker_color=["#28a745", "#ffc107", "#dc3545"],
            name="Compliance"
        ),
        row=1, col=2
    )

    fig.update_layout(height=400, title_text="Risk & Compliance Overview")
    return fig

def create_geospatial_map():
    """Create supplier location map"""
    fig = px.scatter_mapbox(
        df_suppliers,
        lat="Latitude",
        lon="Longitude",
        color="Carbon_Footprint_tCO2",
        size="Contract_Value",
        hover_name="Supplier",
        hover_data=["Region", "ESG_Score", "Risk_Level"],
        color_continuous_scale="RdYlGn_r",
        size_max=15,
        zoom=1,
        title="Global Supplier Carbon Footprint"
    )

    fig.update_layout(
        mapbox_style="open-street-map",
        height=500,
        margin={"r": 0, "t": 30, "l": 0, "b": 0}
    )
    return fig

def generate_alert_summary():
    """Generate alert summary text"""
    alerts = df_suppliers[df_suppliers["Last_Alert"].notna()]
    high_risk = df_suppliers[df_suppliers["Risk_Level"] == "High"]
    overdue_audits = df_suppliers[
        pd.to_datetime(df_suppliers["Audit_Date"]) < (datetime.now() - timedelta(days=180))
    ]

    summary = f"""
🚨 **SUSTAINABILITY ALERTS**

**Active Alerts:** {len(alerts)}
- High Carbon Spikes: {len(alerts[alerts['Last_Alert'] == 'High Carbon Spike'])}
- Overdue Audits: {len(alerts[alerts['Last_Alert'] == 'Audit Overdue'])}
- Compliance Issues: {len(alerts[alerts['Last_Alert'] == 'Compliance Issue'])}

**Risk Assessment:**
- High Risk Suppliers: {len(high_risk)}
- Suppliers Needing Audit: {len(overdue_audits)}

**Key Actions Needed:**
1. Contact {high_risk.iloc[0]['Supplier'] if len(high_risk) > 0 else 'N/A'} - High carbon emissions
2. Schedule audits for {len(overdue_audits)} suppliers
3. Review compliance for suppliers in Red status
    """
    return summary

def simulate_slack_notification():
    """Simulate Slack alert notification"""
    alerts = [
        "🔴 ALERT: GreenTech Manufacturing carbon emissions exceeded threshold by 15%",
        "⚠️ WARNING: EcoFabrics Ltd audit is 30 days overdue",
        "🔴 CRITICAL: SustainablePack Co labor compliance status changed to RED",
        "⚠️ NOTICE: CleanEnergy Logistics ESG score dropped below 75",
        "🔴 URGENT: CarbonNeutral Shipping reported supply chain disruption"
    ]

    selected_alert = random.choice(alerts)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    notification = f"""
📱 **SLACK NOTIFICATION**
**Channel:** #sustainability-alerts
**Time:** {timestamp}

{selected_alert}

**Actions Available:**
- 📊 View Supplier Dashboard
- 📞 Contact Procurement Team
- 📋 Create CRM Task
- 📈 Generate Compliance Report
    """
    return notification

def create_trends_chart():
    """Create sustainability trends over time"""
    # Generate time series data
    dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='M')

    # Simulate trends for key metrics
    carbon_trend = [100 + np.sin(i/2) * 10 + np.random.normal(0, 5) for i in range(len(dates))]
    esg_trend = [80 + np.cos(i/3) * 5 + np.random.normal(0, 2) for i in range(len(dates))]
    compliance_trend = [85 + np.sin(i/4) * 8 + np.random.normal(0, 3) for i in range(len(dates))]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dates, y=carbon_trend,
        mode='lines+markers',
        name='Avg Carbon Footprint',
        line=dict(color='#dc3545')
    ))

    fig.add_trace(go.Scatter(
        x=dates, y=esg_trend,
        mode='lines+markers',
        name='Avg ESG Score',
        line=dict(color='#28a745')
    ))

    fig.add_trace(go.Scatter(
        x=dates, y=compliance_trend,
        mode='lines+markers',
        name='Compliance Rate %',
        line=dict(color='#007bff')
    ))

    fig.update_layout(
        title="Sustainability Trends (2024)",
        xaxis_title="Date",
        yaxis_title="Score/Rate",
        height=400,
        hovermode='x unified'
    )

    return fig

def filter_suppliers_by_risk(risk_level="All"):
    """Filter and display supplier data by risk level"""
    if risk_level == "All":
        filtered_df = df_suppliers
    else:
        filtered_df = df_suppliers[df_suppliers["Risk_Level"] == risk_level]

    # Select key columns for display
    display_columns = [
        "Supplier", "Region", "Carbon_Footprint_tCO2", "ESG_Score",
        "Risk_Level", "Labor_Compliance", "Last_Alert"
    ]

    return filtered_df[display_columns].head(10)

# Create Gradio interface
def create_dashboard():
    with gr.Blocks(
        title="EcoVision: Sustainable Supply Chain Analytics",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            font-family: 'Segoe UI', sans-serif;
        }
        .tab-nav {
            background: linear-gradient(90deg, #28a745, #20c997);
        }
        """
    ) as demo:

        gr.HTML("""
        <div style="text-align: center; padding: 20px; background: linear-gradient(90deg, #28a745, #20c997); color: white; margin-bottom: 20px;">
            <h1>🌱 EcoVision: Sustainable Supply Chain Analytics</h1>
            <p>Real-time ESG monitoring and sustainability insights for supply chain excellence</p>
        </div>
        """)

        with gr.Tabs():

            # Overview Dashboard
            with gr.Tab("📊 Overview Dashboard"):
                with gr.Row():
                    with gr.Column(scale=2):
                        region_filter = gr.Dropdown(
                            choices=["All"] + df_suppliers["Region"].unique().tolist(),
                            value="All",
                            label="Filter by Region"
                        )
                        carbon_plot = gr.Plot(label="Carbon Footprint Analysis")

                    with gr.Column(scale=1):
                        alert_summary = gr.Textbox(
                            label="Alert Summary",
                            lines=15,
                            max_lines=20,
                            interactive=False
                        )

                with gr.Row():
                    esg_heatmap = gr.Plot(label="ESG Scorecard")
                    risk_dashboard = gr.Plot(label="Risk Assessment")

            # Geospatial View
            with gr.Tab("🗺️ Global View"):
                geo_map = gr.Plot(label="Supplier Locations & Carbon Impact")

                with gr.Row():
                    with gr.Column():
                        gr.HTML("<h3>Key Insights</h3>")
                        gr.HTML("""
                        <ul>
                            <li>🔴 Larger circles indicate higher contract values</li>
                            <li>🟢 Green markers show lower carbon footprint</li>
                            <li>🔴 Red markers indicate high carbon emissions</li>
                            <li>📍 Click markers for supplier details</li>
                        </ul>
                        """)

            # Trends & Analytics
            with gr.Tab("📈 Trends & Analytics"):
                trends_chart = gr.Plot(label="Sustainability Trends Over Time")

                with gr.Row():
                    with gr.Column():
                        risk_filter = gr.Dropdown(
                            choices=["All", "High", "Medium", "Low"],
                            value="All",
                            label="Filter by Risk Level"
                        )
                    with gr.Column():
                        refresh_data = gr.Button("🔄 Refresh Data", variant="secondary")

                supplier_table = gr.Dataframe(
                    label="Supplier Details",
                    headers=["Supplier", "Region", "Carbon (tCO2)", "ESG Score", "Risk", "Compliance", "Alert"],
                    wrap=True
                )

            # Alerts & Notifications
            with gr.Tab("🚨 Alerts & Actions"):
                with gr.Row():
                    with gr.Column():
                        slack_notification = gr.Textbox(
                            label="Latest Slack Notification",
                            lines=10,
                            max_lines=15,
                            interactive=False
                        )
                        generate_alert_btn = gr.Button("📱 Generate New Alert", variant="primary")

                    with gr.Column():
                        gr.HTML("<h3>🎯 Action Center</h3>")
                        with gr.Group():
                            gr.Button("📞 Contact High-Risk Suppliers", size="lg")
                            gr.Button("📊 Generate ESG Report", size="lg")
                            gr.Button("📅 Schedule Audits", size="lg")
                            gr.Button("📈 Export Analytics", size="lg")
                            gr.Button("⚙️ Update Thresholds", size="lg")

        # Event handlers
        def update_carbon_chart(region):
            return create_carbon_footprint_chart(region)

        def update_supplier_table(risk):
            return filter_suppliers_by_risk(risk)

        def refresh_all_data():
            global df_suppliers
            df_suppliers = generate_supplier_data()
            return (
                create_carbon_footprint_chart("All"),
                generate_alert_summary(),
                create_esg_scorecard(),
                create_risk_dashboard(),
                create_geospatial_map(),
                create_trends_chart(),
                filter_suppliers_by_risk("All")
            )

        # Connect events
        region_filter.change(
            fn=update_carbon_chart,
            inputs=[region_filter],
            outputs=[carbon_plot]
        )

        risk_filter.change(
            fn=update_supplier_table,
            inputs=[risk_filter],
            outputs=[supplier_table]
        )

        generate_alert_btn.click(
            fn=simulate_slack_notification,
            outputs=[slack_notification]
        )

        refresh_data.click(
            fn=refresh_all_data,
            outputs=[carbon_plot, alert_summary, esg_heatmap, risk_dashboard, geo_map, trends_chart, supplier_table]
        )

        # Initialize dashboard
        demo.load(
            fn=lambda: (
                create_carbon_footprint_chart("All"),
                generate_alert_summary(),
                create_esg_scorecard(),
                create_risk_dashboard(),
                create_geospatial_map(),
                create_trends_chart(),
                filter_suppliers_by_risk("All"),
                simulate_slack_notification()
            ),
            outputs=[carbon_plot, alert_summary, esg_heatmap, risk_dashboard, geo_map, trends_chart, supplier_table, slack_notification]
        )

    return demo

# Launch the application
if __name__ == "__main__":
    demo = create_dashboard()
    demo.launch(
        share=True,  # Creates public link for sharing
        server_name="0.0.0.0",  # Allows external access
        server_port=7860,  # Default Gradio port
        show_error=True,
        debug=True
    )

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1eb270a24e73b12542.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-1038660216.py:225: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/tmp/ipython-input-1038660216.py:225: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

